In [19]:
import string
from pathlib import Path
from collections import defaultdict
import json
import numpy as np
import math
file = open('./input_sample.txt', 'r')

for line in file.readlines():
    line = line.replace('\n','')
    jets = [1 if l=='>' else -1 for l in line]
    print(f'readed jets line')

print(len(jets))
print(sum(jets))

readed jets line
40
2


In [20]:
cave = np.zeros((7,6000))
highest_point = 0
num_rocks_stopped = 0
rock_index = 0
is_stopped = True
rock_types = {
    0: [2, 5, np.array([0,0,0,0]), np.array([1,1,1,1])],
    1: [2, 4, np.array([1,0,1]), np.array([2,3,2])],
    2: [2, 4, np.array([0,0,0]), np.array([1,1,3])],
    3: [2, 2, np.array([0]), np.array([4])],
    4: [2, 3, np.array([0,0]), np.array([2,2])],
}
rock_shapes = {}
for idx, rock in rock_types.items():
    x1, x2, min_heights, heights = rock
    width = x2-x1+1
    height = max(heights)
    rock_shape = np.zeros((width,height))
    for i in range(width):
        rock_shape[i,min_heights[i]:heights[i]] = 1
    rock_shapes[idx] = rock_shape

print(rock_shapes)

{0: array([[1.],
       [1.],
       [1.],
       [1.]]), 1: array([[0., 1., 0.],
       [1., 1., 1.],
       [0., 1., 0.]]), 2: array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 1., 1.]]), 3: array([[1., 1., 1., 1.]]), 4: array([[1., 1.],
       [1., 1.]])}


In [21]:
def can_move(cave, move_x, rock_shape, x1, x2, y_pos):
    y1 = int(min(y_pos))
    old_window = cave[x1:x2+1, y1:y1+rock_shape.shape[1]]
    window = cave[x1+move_x:x2+move_x+1, y1:y1+rock_shape.shape[1]]
    assert window.shape == rock_shape.shape
    ans = not (np.bitwise_and(window>0, rock_shape>0).any())

    # if (np.bitwise_and(old_window>0, rock_shape>0).any()):
    #     print(f'rock is on another rock moving l/r??')
    #     visible_cave = cave.copy()
    #     visible_cave[x1:x2+1, min(y_pos):min(y_pos)+rock_shape.shape[1]] += 2*rock_shape
    #     print(f'with cave {visible_cave[:,:int(max(y_pos)+5)].T[::-1,:]}')
    #     print(f'with y_pos : {y_pos}')
    #     print(f'with x1, x2 : {x1},{x2}')
    #     print(f'with shape {rock_shape.T[::-1,:]}')
    #     print(f'window is {window.T[::-1,:]}')
    #     print(f'can move is {ans}')
    #     assert False

    return ans

def can_move_down(cave, rock_shape, x1, x2, y_pos):

    y1 = int(min(y_pos))
    if y1==0:
        return False
    old_window = cave[x1:x2+1, y1:y1+rock_shape.shape[1]]
    window = cave[x1:x2+1, y1-1:y1-1+rock_shape.shape[1]]
    assert window.shape == rock_shape.shape
    ans = not (np.bitwise_and(window>0, rock_shape>0).any())

    # if (np.bitwise_and(old_window>0, rock_shape>0).any()):
    #     print(f'rock is on another rock going down??')
    #     visible_cave = cave.copy()
    #     visible_cave[x1:x2+1, min(y_pos):min(y_pos)+rock_shape.shape[1]] += 2*rock_shape
    #     print(f'with cave {visible_cave[:,:int(max(y_pos)+5)].T[::-1,:]}')
    #     print(f'with old window : {old_window}')
    #     print(f'with y_pos : {y_pos}')
    #     print(f'with x1, x2 : {x1},{x2}')
    #     print(f'with shape {rock_shape.T[::-1,:]}')
    #     print(f'window is {window.T[::-1,:]}')
    #     print(f'can move is {ans}')
    #     assert False

    return ans

In [22]:
jet_idx = -1
while num_rocks_stopped<2022:
    jet_idx = (jet_idx+1)%len(jets)
    if is_stopped:
        is_stopped = False
        current_rock = rock_types[rock_index].copy()
        rock_shape = rock_shapes[rock_index]
        x1, x2, min_heights, heights = current_rock
        y_pos = min_heights+highest_point+3

    move_x = jets[jet_idx]
    if move_x>0:
        if x2<6 and can_move(cave, move_x, rock_shape, x1, x2, y_pos):
            x1+=1
            x2+=1
    else:
        if x1>0 and can_move(cave, move_x, rock_shape, x1, x2, y_pos):
            x1+=-1
            x2+=-1
    
    movable = can_move_down(cave, rock_shape, x1, x2, y_pos)

    # visible_cave = cave.copy()
    # visible_cave[x1:x2+1, min(y_pos):min(y_pos)+rock_shape.shape[1]] = 2*rock_shape
    # print(f'movable down is {movable}')
    # print(f'highest point is {highest_point}')
    # print(f'y pos is {y_pos}')
    # print(f'CURRENT CAVE : \n {visible_cave[:,:highest_point+5].T[::-1,:]}')
    
    if movable:
        y_pos = y_pos-1
    else:
        bottom = min(y_pos)
        is_stopped = True
        rock_index = (rock_index+1)%5
        num_rocks_stopped += 1

        cave[x1:x2+1, bottom:bottom+rock_shape.shape[1]] = rock_shape
        xs, ys = np.where(cave>0)
        highest_point = max(ys)+1
        print(f'cave after stopping : {cave[:,:highest_point+2].T[::-1,:]}')
        print(f'highest point {highest_point}')
        print(f'num moves = {jet_idx+1}')
        print(f'stopping {num_rocks_stopped}th rock {rock_index%5} at h {bottom}')

cave after stopping : [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0.]]
highest point 1
num moves = 4
stopping 1th rock 1 at h 0
cave after stopping : [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0.]]
highest point 4
num moves = 8
stopping 2th rock 2 at h 1
cave after stopping : [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0.]]
highest point 6
num moves = 13
stopping 3th rock 3 at h 3
cave after stopping : [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0.]]
highest point 7
num moves = 20
stopping 4th rock 4 at h 3
cave after stopping : [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

In [23]:
print(highest_point)
print(num_rocks_stopped)
xs, ys = np.where(cave>0)
print(max(ys))

3068
2022
3067


In [24]:
[3079. 3080. 3079. 3079. 3079. 3079. 3079.]
3080.0

SyntaxError: invalid syntax (<ipython-input-24-02d889edeea8>, line 1)